In [ ]:
#STEP1-Data Collection & HTML Parsing

In [446]:
import pandas as pd

# Read the dataset
df = pd.read_csv('seo-content-detector/data/data.csv')
df.head()


,url,html_content
0,https://www.cm-alliance.com/cybersecurity-blog,"<!doctype html><!--[if lt IE 7]> <html class=""..."
1,https://www.varonis.com/blog/cybersecurity-tips,"<!doctype html><html lang=""en""><head>\n <me..."
2,https://www.cisecurity.org/insights/blog/11-cy...,<!DOCTYPE html><html data-unhead-vue-server-re...
3,https://www.cisa.gov/topics/cybersecurity-best...,"\n\n<!DOCTYPE html>\n<html lang=""en"" dir=""ltr""..."
4,https://www.qnbtrust.bank/Resources/Learning-C...,NaN


In [276]:
from bs4 import BeautifulSoup

def extract_text_features(html):
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.title.string if soup.title else ""
    # Extract body text from common content tags
    body_elements = soup.find_all(['article', 'main', 'p'])
    body_text = " ".join([elem.get_text(separator=" ", strip=True) for elem in body_elements])
    if not body_text:
        body_text = soup.get_text(separator=" ", strip=True)
    wordcount = len(body_text.split())
    return title, body_text, wordcount

features = []
for idx, row in df.iterrows():
    try:
        title, body_text, wordcount = extract_text_features(row['htmlcontent'])
        features.append({'url': row['url'], 'title': title, 'bodytext': body_text, 'wordcount': wordcount})
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

features_df = pd.DataFrame(features)
features_df.to_csv('data/extractedcontent.csv', index=False)


Error processing row 0: 'htmlcontent'
Error processing row 1: 'htmlcontent'
Error processing row 2: 'htmlcontent'
Error processing row 3: 'htmlcontent'
Error processing row 4: 'htmlcontent'
Error processing row 5: 'htmlcontent'
Error processing row 6: 'htmlcontent'
Error processing row 7: 'htmlcontent'
Error processing row 8: 'htmlcontent'
Error processing row 9: 'htmlcontent'
Error processing row 10: 'htmlcontent'
Error processing row 11: 'htmlcontent'
Error processing row 12: 'htmlcontent'
Error processing row 13: 'htmlcontent'
Error processing row 14: 'htmlcontent'
Error processing row 15: 'htmlcontent'
Error processing row 16: 'htmlcontent'
Error processing row 17: 'htmlcontent'
Error processing row 18: 'htmlcontent'
Error processing row 19: 'htmlcontent'
Error processing row 20: 'htmlcontent'
Error processing row 21: 'htmlcontent'
Error processing row 22: 'htmlcontent'
Error processing row 23: 'htmlcontent'
Error processing row 24: 'htmlcontent'
Error processing row 25: 'htmlconte

In [277]:
import pandas as pd

df = pd.read_csv('data/data.csv')
print(df.columns)


Index(['url', 'html_content'], dtype='object')


In [278]:
from bs4 import BeautifulSoup

def extract_text_features(html):
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.title.string if soup.title else ""
    body_elements = soup.find_all(['article', 'main', 'p'])
    body_text = " ".join([elem.get_text(separator=" ", strip=True) for elem in body_elements])
    if not body_text:
        body_text = soup.get_text(separator=" ", strip=True)
    wordcount = len(body_text.split())
    return title, body_text, wordcount

features = []
for idx, row in df.iterrows():
    try:
        title, body_text, wordcount = extract_text_features(row['html_content'])
        features.append({'url': row['url'], 'title': title, 'bodytext': body_text, 'wordcount': wordcount})
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

features_df = pd.DataFrame(features)
features_df.to_csv('extractedcontent.csv', index=False)


Error processing row 4: object of type 'float' has no len()
Error processing row 12: object of type 'float' has no len()
Error processing row 18: object of type 'float' has no len()
Error processing row 20: object of type 'float' has no len()
Error processing row 23: object of type 'float' has no len()
Error processing row 40: object of type 'float' has no len()
Error processing row 41: object of type 'float' has no len()
Error processing row 51: object of type 'float' has no len()
Error processing row 55: object of type 'float' has no len()
Error processing row 75: object of type 'float' has no len()
Error processing row 76: object of type 'float' has no len()
Error processing row 77: object of type 'float' has no len()


In [279]:

print(df['html_content'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0).value_counts())


html_content
True     69
False    12
Name: count, dtype: int64


In [280]:
from bs4 import BeautifulSoup

def extract_text_features(html):
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.title.string if soup.title else ""
    body_elements = soup.find_all(['article', 'main', 'p'])
    body_text = " ".join([elem.get_text(separator=" ", strip=True) for elem in body_elements])
    if not body_text:
        body_text = soup.get_text(separator=" ", strip=True)
    wordcount = len(body_text.split())
    return title, body_text, wordcount

features = []
for idx, row in df.iterrows():
    try:
        html = row['html_content']
        # Skip if not a string or is empty
        if not isinstance(html, str) or not html.strip():
            continue
        title, body_text, wordcount = extract_text_features(html)
        features.append({'url': row['url'], 'title': title, 'bodytext': body_text, 'wordcount': wordcount})
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        


# Save output
features_df = pd.DataFrame(features)
features_df.to_csv('extractedcontent.csv', index=False)
print(f"Number of rows processed: {len(features)}")
features_df = pd.DataFrame(features)
features_df.to_csv('extractedcontent.csv', index=False)


Number of rows processed: 69


In [281]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Load extracted content
df = pd.read_csv('extractedcontent.csv')

# Download NLTK data if needed
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<[^>]+>', ' ', text)  # Remove HTML tags if any
    text = re.sub(r'\W+', ' ', text)  # Remove non-alphanumeric characters
    words = word_tokenize(text)
    words = [ps.stem(w) for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(words)

df['clean_bodytext'] = df['bodytext'].astype(str).apply(preprocess_text)
df.to_csv('preprocessed_content.csv', index=False)
print(df[['url', 'clean_bodytext']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kpraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kpraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                 url  \
0     https://www.cm-alliance.com/cybersecurity-blog   
1    https://www.varonis.com/blog/cybersecurity-tips   
2  https://www.cisecurity.org/insights/blog/11-cy...   
3  https://www.cisa.gov/topics/cybersecurity-best...   
4  https://nordlayer.com/learn/network-security/b...   

                                      clean_bodytext  
0  cyber crisi tabletop exercis cyber secur awar ...  
1  data secur platform buy capabl data discoveri ...  
2  home insight blog post cyber defens tip stay s...  
3  offici websit unit state govern know offici we...  
4  cut edg busi vpn holist approach secur connect...  


In [ ]:
#STEP2-Text Preprocessing & Feature Engineering

In [282]:
def count_words(text):
    return len(text.split())

def title_length(text):
    return len(text.split())

df['clean_title'] = df['title'].astype(str).apply(preprocess_text)
df['title_wordcount'] = df['clean_title'].apply(count_words)
df['body_wordcount'] = df['clean_bodytext'].apply(count_words)

# Save augmented dataset
df.to_csv('features_engineered.csv', index=False)
print(df[['url', 'title_wordcount', 'body_wordcount']].head())


                                                 url  title_wordcount  \
0     https://www.cm-alliance.com/cybersecurity-blog                3   
1    https://www.varonis.com/blog/cybersecurity-tips                7   
2  https://www.cisecurity.org/insights/blog/11-cy...                7   
3  https://www.cisa.gov/topics/cybersecurity-best...                8   
4  https://nordlayer.com/learn/network-security/b...                5   

   body_wordcount  
0             234  
1            3284  
2            1098  
3             989  
4            2756  


In [283]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load preprocessed data
df = pd.read_csv('features_engineered.csv')

# Vectorize the clean body text using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['clean_bodytext'])

# Calculate pairwise cosine similarity matrix
cos_sim_matrix = cosine_similarity(tfidf_matrix)

# To demonstrate, find duplicates for the first document with similarity > 0.8
threshold = 0.8
first_doc_sim = cos_sim_matrix[0]
duplicates_idx = np.where(first_doc_sim > threshold)[0]

print(f"Documents similar to the first URL ({df.loc[0, 'url']}):")
for idx in duplicates_idx:
    if idx != 0:
        print(f" - {df.loc[idx, 'url']} with similarity {first_doc_sim[idx]:.2f}")


Documents similar to the first URL (https://www.cm-alliance.com/cybersecurity-blog):


In [284]:
from scipy.cluster.hierarchy import linkage, fcluster
import pandas as pd

# Use precomputed cosine similarity matrix (cos_sim_matrix)
# Convert similarity matrix to distance matrix (distance = 1 - similarity)
distance_matrix = 1 - cos_sim_matrix

# Perform hierarchical clustering using 'average' linkage
Z = linkage(distance_matrix, method='average')

# Create cluster labels with a threshold for maximum distance
max_distance = 0.2  # Corresponds to similarity cutoff of 0.8
clusters = fcluster(Z, t=max_distance, criterion='distance')

# Add cluster labels to your dataframe
df['duplicate_cluster'] = clusters

# Example: Show URLs and their cluster IDs
print(df[['url', 'duplicate_cluster']].head(20))

# Save with clusters
df.to_csv('features_with_clusters.csv', index=False)


                                                  url  duplicate_cluster
0      https://www.cm-alliance.com/cybersecurity-blog                 50
1     https://www.varonis.com/blog/cybersecurity-tips                 29
2   https://www.cisecurity.org/insights/blog/11-cy...                 30
3   https://www.cisa.gov/topics/cybersecurity-best...                 51
4   https://nordlayer.com/learn/network-security/b...                 24
5   https://www.fortinet.com/resources/cyberglossa...                 25
6   https://www.cisco.com/site/us/en/learn/topics/...                 27
7   https://www.trendmicro.com/en_us/what-is/netwo...                 26
8   https://digitdefence.com/blog/fundamentals-of-...                 28
9   https://guardiandigital.com/resources/blog/gui...                 52
10                           https://cofense.com/blog                 54
11             https://www.phriendlyphishing.com/blog                 55
12  https://inspiredelearning.com/blog/phishing-pr.

In [ ]:
#STEP3-Duplicate Detection 

In [285]:
!pip install textstat


In [286]:
!pip install sentence-transformers


In [287]:
!pip install tf-keras




In [288]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load preprocessed content CSV (with clean body text)
df = pd.read_csv('preprocessed_content.csv')

# Generate TF-IDF matrix as embedding representation
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_bodytext'])

# Store embeddings as lists
df['embedding'] = list(tfidf_matrix.toarray())

# Save updated dataframe
df.to_csv('text_features_with_tfidf_embeddings.csv', index=False)

print(df[['url', 'embedding']].head())


                                                 url  \
0     https://www.cm-alliance.com/cybersecurity-blog   
1    https://www.varonis.com/blog/cybersecurity-tips   
2  https://www.cisecurity.org/insights/blog/11-cy...   
3  https://www.cisa.gov/topics/cybersecurity-best...   
4  https://nordlayer.com/learn/network-security/b...   

                                           embedding  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.014283106547502242, 0.0, 0.0, 0.0, 0.0, 0.0...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01458775...  


In [289]:
print(df.columns)


Index(['url', 'title', 'bodytext', 'wordcount', 'clean_bodytext', 'embedding'], dtype='object')


In [290]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast
import os

# Make sure the data folder exists
os.makedirs('data', exist_ok=True)

# Load your features CSV - update the filename if needed
df = pd.read_csv('data/text_features_with_tfidf_embeddings.csv')

# Convert embedding strings to numpy arrays, ignore rows where parsing fails (in case of ellipsis, etc.)
def parse_embedding(x):
    try:
        return np.array(ast.literal_eval(x))
    except:
        return np.zeros(10)  # or appropriate size

df['embedding'] = df['embedding'].apply(parse_embedding)
embedding_matrix = np.vstack(df['embedding'].values)

# Compute cosine similarity matrix
cos_sim_matrix = cosine_similarity(embedding_matrix)

# Set similarity threshold
similarity_threshold = 0.80

# Find duplicate pairs (upper triangle only)
duplicate_pairs = []
num_docs = len(df)
for i in range(num_docs):
    for j in range(i + 1, num_docs):
        sim = cos_sim_matrix[i, j]
        if sim > similarity_threshold:
            duplicate_pairs.append((df.loc[i, 'url'], df.loc[j, 'url'], sim))

duplicates_df = pd.DataFrame(duplicate_pairs, columns=['url1', 'url2', 'similarity'])

# Thin content detection (word count < 500)
df['is_thin'] = df['wordcount'] < 500

# Report
total_pages = num_docs
num_duplicate_pairs = len(duplicate_pairs)
num_thin_pages = df['is_thin'].sum()
percent_thin = (num_thin_pages / total_pages) * 100

print(f"Total pages analyzed: {total_pages}")
print(f"Duplicate pairs found: {num_duplicate_pairs}")
print(f"Thin content pages: {num_thin_pages} ({percent_thin:.1f}%)")

# Save all outputs to the data folder
duplicates_df.to_csv('data/duplicates.csv', index=False)
df.to_csv('data/text_features_with_tfidf_embeddings_with_thin.csv', index=False)


Total pages analyzed: 69
Duplicate pairs found: 0
Thin content pages: 14 (20.3%)


In [ ]:
#STEP-4: Content Quality Scoring

In [388]:
# ---- Safe, consistent model comparison ----
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Ensure features variable matches your df column names
features = ['wordcount', 'sentence_count', 'flesch_reading_ease', 'avg_sentence_length', 'lexical_diversity']
X = df[features].fillna(0)
y = le.transform(df['quality_label'])  # use your existing LabelEncoder

# Train/test are already defined (X_train, X_test, y_train, y_test)
# But ensure they come from X,y above. If not, recreate:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Define models as pipelines so every model accepts raw X
models = {
    "LogisticRegression": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))]),
    "SVC": Pipeline([("scaler", StandardScaler()), ("clf", SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42))]),
    "KNN": Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier(n_neighbors=5))]),
    "GaussianNB": Pipeline([("scaler", StandardScaler()), ("clf", GaussianNB())]),
    "RandomForest": Pipeline([("clf", RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42))]),
    "ExtraTrees": Pipeline([("clf", ExtraTreesClassifier(n_estimators=200, class_weight='balanced', random_state=42))]),
    "GradientBoosting": Pipeline([("clf", GradientBoostingClassifier(n_estimators=200, random_state=42))]),
    "AdaBoost": Pipeline([("clf", AdaBoostClassifier(n_estimators=200, random_state=42))])
}

results = []
fitted_models = {}
for name, pipe in models.items():
    pipe.fit(X_train, y_train)                      # ALWAYS pass raw X_train
    y_pred = pipe.predict(X_test)                   # ALWAYS pass raw X_test
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average='macro')
    results.append({"Model": name, "Accuracy": round(acc,3), "Macro F1": round(f1m,3)})
    fitted_models[name] = pipe

    print("\n" + "="*50)
    print(f"Model: {name} — Accuracy: {acc:.3f}  Macro F1: {f1m:.3f}")
    print("="*50)
    print(classification_report(y_test, y_pred, target_names=le.classes_, digits=3))
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)
    print("Confusion matrix (rows=true, cols=pred):")
    display(cm_df)

summary_df = pd.DataFrame(results).sort_values(by="Macro F1", ascending=False).reset_index(drop=True)
print("\nSummary:")
display(summary_df)



Model: LogisticRegression — Accuracy: 0.762  Macro F1: 0.760
              precision    recall  f1-score   support

        High      0.714     0.833     0.769         6
         Low      1.000     0.600     0.750         5
      Medium      0.727     0.800     0.762        10

    accuracy                          0.762        21
   macro avg      0.814     0.744     0.760        21
weighted avg      0.788     0.762     0.761        21

Confusion matrix (rows=true, cols=pred):


,High,Low,Medium
High,5,0,1
Low,0,3,2
Medium,2,0,8



Model: SVC — Accuracy: 0.667  Macro F1: 0.638
              precision    recall  f1-score   support

        High      0.571     0.667     0.615         6
         Low      1.000     0.400     0.571         5
      Medium      0.667     0.800     0.727        10

    accuracy                          0.667        21
   macro avg      0.746     0.622     0.638        21
weighted avg      0.719     0.667     0.658        21

Confusion matrix (rows=true, cols=pred):


,High,Low,Medium
High,4,0,2
Low,1,2,2
Medium,2,0,8



Model: KNN — Accuracy: 0.476  Macro F1: 0.357
              precision    recall  f1-score   support

        High      0.400     0.667     0.500         6
         Low      0.000     0.000     0.000         5
      Medium      0.545     0.600     0.571        10

    accuracy                          0.476        21
   macro avg      0.315     0.422     0.357        21
weighted avg      0.374     0.476     0.415        21

Confusion matrix (rows=true, cols=pred):


...

,High,Low,Medium
High,4,2,0
Low,0,1,4
Medium,4,1,5



Model: AdaBoost — Accuracy: 0.429  Macro F1: 0.410
              precision    recall  f1-score   support

        High      0.333     0.333     0.333         6
         Low      0.250     0.400     0.308         5
      Medium      0.714     0.500     0.588        10

    accuracy                          0.429        21
   macro avg      0.433     0.411     0.410        21
weighted avg      0.495     0.429     0.449        21

Confusion matrix (rows=true, cols=pred):


,High,Low,Medium
High,2,4,0
Low,1,2,2
Medium,3,2,5



Summary:


,Model,Accuracy,Macro F1
0,LogisticRegression,0.762,0.760
1,SVC,0.667,0.638
2,RandomForest,0.476,0.440
3,ExtraTrees,0.476,0.440
4,GradientBoosting,0.476,0.440
5,AdaBoost,0.429,0.410
6,KNN,0.476,0.357
7,GaussianNB,0.429,0.278


In [ ]:
#Step-5:Real-Time Analysis Demo 

In [444]:
# Improved interactive + upload runner cell (safe, with fallbacks)
import json, io, re, traceback, sys
from IPython.display import display, clear_output

# Ensure pandas, etc are imported (assumes analyze_url already defined and uses pandas)
try:
    import pandas as pd
except Exception:
    print("pandas not available - please install it (pip install pandas).")

# ---- URL parsing helpers (same as you used) ----
def _parse_urls_from_text(text):
    lines = [ln.strip() for ln in re.split(r'[\r\n,]+', text) if ln.strip()]
    return lines

def _parse_urls_from_csv_bytes(bytes_content):
    try:
        s = io.StringIO(bytes_content.decode('utf-8', errors='replace'))
        df_in = pd.read_csv(s)
        url_cols = [c for c in df_in.columns if c.lower() == 'url' or 'url' in c.lower()]
        if url_cols:
            return df_in[url_cols[0]].dropna().astype(str).tolist()
        if df_in.shape[1] == 1:
            return df_in.iloc[:,0].dropna().astype(str).tolist()
        return []
    except Exception:
        try:
            text = bytes_content.decode('utf-8', errors='replace')
            return _parse_urls_from_text(text)
        except Exception:
            return []

# ---- Console interactive runner ----
def run_interactive(default_df_name="df"):
    """
    Console prompt: accepts a single URL or local file path containing multiple URLs.
    Uses analyze_url(url, df=...) from the notebook if available.
    """
    df_to_use = globals().get(default_df_name, None)
    try:
        print("Enter a single URL, or a path to a local file containing URLs (newline-separated or CSV).")
        user_in = input("URL or file path (leave empty to cancel): ").strip()
        if not user_in:
            print("No input provided. Exiting.")
            return None

        # Gather URLs
        if user_in.startswith("http://") or user_in.startswith("https://"):
            urls = [user_in]
        else:
            try:
                with open(user_in, "rb") as f:
                    content = f.read()
                urls = _parse_urls_from_csv_bytes(content)
                if not urls:
                    urls = _parse_urls_from_text(content.decode('utf-8', errors='replace'))
                if not urls:
                    print("No URLs found in file; treating input as literal URL.")
                    urls = [user_in]
            except FileNotFoundError:
                urls = [user_in]

        print(f"Analyzing {len(urls)} URL(s)...")
        results = []
        for u in urls:
            print(f"\n--- {u} ---")
            try:
                res = analyze_url(u, df=df_to_use)
                print(json.dumps(res, indent=2))
                results.append(res)
            except Exception as e:
                print("Error analyzing:", e)
                traceback.print_exc()
                results.append({"url": u, "error": str(e)})
        return results
    except Exception as e:
        print("run_interactive failed:", e)
        traceback.print_exc()
        return None

# ---- Try to enable widget UI; if not available, don't crash ----
WIDGETS_AVAILABLE = True
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception as e:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available. To use the notebook GUI, install ipywidgets and enable it.")
    print("You can install with: pip install ipywidgets")
    print("Then restart the kernel and run this cell again. Falling back to console run_interactive().")

# ---- Widget-based UI with upload support (only if ipywidgets available) ----
if WIDGETS_AVAILABLE:
    def run_widget_with_upload(default_df_name="df"):
        """
        Jupyter widget UI: text box + Analyze button and FileUpload for bulk URLs.
        Uses analyze_url(url, df=...) from the notebook if available.
        """
        txt = widgets.Text(value='', placeholder='Enter URL here', description='URL:', layout=widgets.Layout(width='70%'))
        analyze_btn = widgets.Button(description='Analyze URL', button_style='primary')
        upload = widgets.FileUpload(accept='.txt,.csv', multiple=False, description='Upload file')
        analyze_file_btn = widgets.Button(description='Analyze Uploaded', button_style='success')
        out = widgets.Output(layout={'border': '1px solid #ddd', 'max_height': '400px', 'overflow': 'auto'})

        def analyze_single_url(url):
            df_to_use = globals().get(default_df_name, None)
            try:
                res = analyze_url(url, df=df_to_use)
                return res
            except Exception as e:
                return {"url": url, "error": str(e)}

        def on_click_analyze(b):
            with out:
                clear_output()
                url = txt.value.strip()
                if not url:
                    print("Please enter a URL in the text box.")
                    return
                print(f"Analyzing: {url}")
                try:
                    res = analyze_single_url(url)
                    print(json.dumps(res, indent=2))
                except Exception as e:
                    print("Error:", e)
                    traceback.print_exc()

        def on_click_file(b):
            with out:
                clear_output()
                if not upload.value:
                    print("Please upload a .txt or .csv file containing URLs.")
                    return
                key = list(upload.value.keys())[0]
                content = upload.value[key]['content']
                fname = key
                print(f"Processing uploaded file: {fname}")
                urls = []
                if fname.lower().endswith('.csv'):
                    urls = _parse_urls_from_csv_bytes(content)
                else:
                    try:
                        text = content.decode('utf-8', errors='replace')
                        urls = _parse_urls_from_text(text)
                    except Exception:
                        urls = _parse_urls_from_csv_bytes(content)
                if not urls:
                    print("No URLs found in uploaded file.")
                    return
                print(f"Found {len(urls)} URL(s). Running analysis...")
                df_to_use = globals().get(default_df_name, None)
                for u in urls:
                    print(f"\n--- {u} ---")
                    try:
                        res = analyze_url(u, df=df_to_use)
                        print(json.dumps(res, indent=2))
                    except Exception as e:
                        print("Error analyzing:", e)
                        traceback.print_exc()
                print("\nDone.")

        analyze_btn.on_click(on_click_analyze)
        analyze_file_btn.on_click(on_click_file)

        ui = widgets.VBox([
            widgets.HBox([txt, analyze_btn]),
            widgets.HBox([upload, analyze_file_btn]),
            out
        ])
        display(ui)
else:
    # Provide a stub that tells the user to install ipywidgets
    def run_widget_with_upload(*args, **kwargs):
        print("ipywidgets is not installed or available in this environment.")
        print("Install it with: pip install ipywidgets and restart the kernel, then re-run this cell.")
        return None
# Option 1: create df_text that analyze_url expects, then run analysis
# Assumes your original dataframe is called `df`

# 1. Create a 'text' column from best available field
df_for_similarity = df.copy()
# prefer 'clean_bodytext' if present, else 'bodytext', else 'body_text' etc.
if 'clean_bodytext' in df_for_similarity.columns:
    df_for_similarity['text'] = df_for_similarity['clean_bodytext'].astype(str)
elif 'bodytext' in df_for_similarity.columns:
    df_for_similarity['text'] = df_for_similarity['bodytext'].astype(str)
elif 'body_text' in df_for_similarity.columns:
    df_for_similarity['text'] = df_for_similarity['body_text'].astype(str)
else:
    # fallback: try to assemble from title + body if available
    if 'title' in df_for_similarity.columns and 'bodytext' in df_for_similarity.columns:
        df_for_similarity['text'] = (df_for_similarity['title'].fillna('') + '. ' + df_for_similarity['bodytext'].fillna('')).astype(str)
    else:
        raise RuntimeError("Could not find a suitable text column in df; expected 'clean_bodytext' or 'bodytext' or 'title'+'bodytext'.")

# 2. Quick sanity checks
print("Prepared df_for_similarity with columns:", df_for_similarity.columns.tolist())
print("Non-empty text rows:", df_for_similarity['text'].str.strip().astype(bool).sum(), "of", len(df_for_similarity))

# 3. Run analyze_url with this df (lower threshold so you see candidates)
res = analyze_url(
    "https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e",
    df=df_for_similarity,
    top_k=5,
    similarity_threshold=0.35,
    verbose=True
)

import json
print(json.dumps(res, indent=2))

def analyze_url(url, df=None, top_k=3, similarity_threshold=0.6, timeout=10, verbose=False):
    """
    Scrape URL, extract features, assign quality label, and find similar docs in df.
    - df: optional pandas.DataFrame with a 'text' column (and optional 'url' column).
    - top_k: maximum number of similar matches to return
    - similarity_threshold: minimum cosine similarity to report a match (0-1)
    - verbose: if True prints debug info to stdout
    Returns a dict suitable for json.dumps(...)
    """
    result = {"url": url, "error": None}
    headers = {"User-Agent": "Mozilla/5.0 (compatible; AnalyzerBot/1.0)"}
    try:
        resp = requests.get(url, headers=headers, timeout=timeout)
        resp.raise_for_status()
        html = resp.text
    except Exception as e:
        result["error"] = f"Failed to fetch URL: {e}"
        return result

    text = _extract_main_text(html, url)
    title = None
    try:
        soup = BeautifulSoup(html, "html.parser")
        if soup.title and soup.title.string:
            title = soup.title.string.strip()
    except Exception:
        title = None

    wc = _word_count(text)
    scount = _sentence_count(text)
    try:
        readability = float(textstat.flesch_reading_ease(text)) if text else 0.0
    except Exception:
        readability = 0.0

    avg_sent_len = wc / scount if scount else 0.0
    lex_div = _lexical_diversity(text)

    quality_label = _assign_quality_label(wc, readability)
    is_thin = (wc < 300) or (quality_label == "Low")

    result.update({
        "url": url,
        "title": title,
        "word_count": int(wc),
        "sentence_count": int(scount),
        "avg_sentence_length": float(round(avg_sent_len, 3)),
        "readability": float(round(readability, 3)),
        "lexical_diversity": float(round(lex_div, 3)),
        "quality_label": quality_label,
        "is_thin": bool(is_thin),
        "similar_to": [],
        "similar_to_above_threshold": []
    })

    # -------- Improved similarity logic ----------
    try:
        notes = []
        if isinstance(df, pd.DataFrame) and 'text' in df.columns and len(df) > 0:
            corpus_series = df['text'].fillna("").astype(str)
            if corpus_series.str.strip().eq("").all():
                notes.append("Corpus 'text' column is present but all values are empty.")
            corpus_texts = corpus_series.tolist()

            if 'url' in df.columns:
                urls = df['url'].fillna("").astype(str).tolist()
            else:
                urls = [f"<row_{i}>" for i in range(len(corpus_texts))]

            nonempty_idx = [i for i, t in enumerate(corpus_texts) if t.strip() != ""]
            if not nonempty_idx:
                notes.append("No non-empty corpus documents to compare against.")
            else:
                corpus_texts_nonempty = [corpus_texts[i] for i in nonempty_idx]
                urls_nonempty = [urls[i] for i in nonempty_idx]

                vect = TfidfVectorizer(max_features=5000, stop_words='english')
                X_corpus = vect.fit_transform(corpus_texts_nonempty)
                X_doc = vect.transform([text])

                if X_doc.nnz == 0:
                    notes.append("Document vector is empty after TF-IDF (too short or no overlapping vocabulary). Similarity will be zero.")
                    sims = np.zeros(len(urls_nonempty))
                else:
                    sims = cosine_similarity(X_doc, X_corpus).flatten()

                matches_all = []
                for idx_local, sim in enumerate(sims):
                    orig_idx = nonempty_idx[idx_local]
                    matches_all.append({
                        "url": urls_nonempty[idx_local],
                        "similarity": float(round(float(sim), 4)),
                        "index": int(orig_idx)
                    })

                matches_all_sorted = sorted(matches_all, key=lambda x: x["similarity"], reverse=True)
                result["similar_to"] = matches_all_sorted[:top_k]
                result["similar_to_above_threshold"] = [m for m in matches_all_sorted if m["similarity"] >= similarity_threshold][:top_k]

                if not result["similar_to_above_threshold"]:
                    notes.append(f"No matches above threshold={similarity_threshold}. Returning top {top_k} matches with scores.")
        else:
            notes.append("No DataFrame with a 'text' column was provided for similarity checks.")

        if notes:
            result.setdefault("notes", []).extend(notes)
    except Exception as e:
        result.setdefault("notes", [])
        result["notes"].append(f"Similarity check failed: {e}")
        result["similar_to"] = []
        result["similar_to_above_threshold"] = []

    if verbose:
        print("DEBUG analyze_url:", result.get("notes", "no notes"))
    return result


# ---- Quick note: user might need to restart kernel after installing ipywidgets ----
print("\nInteractive runners ready.")
print("- To use console prompt: run_interactive()")
if WIDGETS_AVAILABLE:
    print("- To use notebook UI with upload: run_widget_with_upload()")
else:
    print("- Notebook UI not available. Install ipywidgets and restart kernel to enable run_widget_with_upload().")


Prepared df_for_similarity with columns: ['url', 'title', 'bodytext', 'wordcount', 'clean_bodytext', 'embedding', 'is_thin', 'flesch_reading_ease', 'sentence_count', 'avg_sentence_length', 'lexical_diversity', 'quality_label', 'text']
Non-empty text rows: 69 of 69
DEBUG analyze_url: ['No matches above threshold=0.35. Returning top 5 matches with scores.']
{
  "url": "https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e",
  "error": null,
  "title": "Natural Language Processing is Fun! | by Adam Geitgey | Medium",
  "word_count": 4096,
  "sentence_count": 191,
  "avg_sentence_length": 21.445,
  "readability": 57.29,
  "lexical_diversity": 0.313,
  "quality_label": "High",
  "is_thin": false,
  "similar_to": [
    {
      "url": "https://medium.com/@amitvsolutions/machine-learning-101-the-complete-beginners-guide-to-machine-learning-686a30cbcf6b",
      "similarity": 0.2035,
      "index": 30
    },
    {
      "url": "https://www.akkio.com/beginners-guide-to-mach

In [448]:
# streamlit_app/app.py
import streamlit as st
import pandas as pd
import numpy as np
import io
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# local utils
from utils.parser import fetch_html, extract_main_text
from utils.features import extract_features_from_text
from utils.scorer import QualityScorer

APP_DIR = Path(__file__).resolve().parent
ROOT_DIR = APP_DIR.parent

st.set_page_config(page_title="SEO Content Detector", layout="centered")
st.title("SEO Content Quality & Duplicate Detector")

# Sidebar controls
st.sidebar.header("Corpus & Settings")
use_default = st.sidebar.checkbox("Use provided corpus (data/extracted_content.csv)", value=True)
uploaded = st.sidebar.file_uploader("Or upload corpus CSV (.csv with 'url' and 'text' columns)", type=["csv"])
similarity_threshold = st.sidebar.slider("Similarity threshold", 0.0, 1.0, 0.35, 0.05)
top_k = st.sidebar.slider("Top K matches", 1, 10, 5)

# Load or prepare corpus
corpus_df = None
if use_default:
    for cand in ["data/extracted_content.csv", "data/data.csv"]:
        p = ROOT_DIR / cand
        if p.exists():
            try:
                corpus_df = pd.read_csv(p)
                st.sidebar.write(f"Loaded corpus: {cand} ({len(corpus_df)} rows)")
                break
            except Exception as e:
                st.sidebar.warning(f"Failed to load {cand}: {e}")
if uploaded is not None:
    try:
        uploaded_bytes = uploaded.read()
        tmp = pd.read_csv(io.BytesIO(uploaded_bytes))
        corpus_df = tmp
        st.sidebar.write(f"Uploaded corpus ({len(corpus_df)} rows)")
    except Exception as e:
        st.sidebar.error(f"Failed to read uploaded file: {e}")

# Normalize corpus text column name if present
if isinstance(corpus_df, pd.DataFrame):
    if 'text' not in corpus_df.columns:
        if 'clean_bodytext' in corpus_df.columns:
            corpus_df['text'] = corpus_df['clean_bodytext'].astype(str)
        elif 'bodytext' in corpus_df.columns:
            corpus_df['text'] = corpus_df['bodytext'].astype(str)
        elif 'body' in corpus_df.columns:
            corpus_df['text'] = corpus_df['body'].astype(str)
        else:
            st.sidebar.warning("Corpus has no 'text' column — similarity disabled.")
            corpus_df = None

# Initialize scorer (loads quality_model.pkl if present; uses rule fallback otherwise)
scorer = QualityScorer(model_path=APP_DIR / "models" / "quality_model.pkl")

# UI: URL input
st.subheader("Analyze a URL")
url = st.text_input("Enter article URL", "")
analyze = st.button("Analyze")

def build_tfidf(corpus_texts):
    vect = TfidfVectorizer(max_features=20000, ngram_range=(1,2), stop_words='english')
    X = vect.fit_transform(corpus_texts)
    return vect, X

if analyze:
    if not url.strip():
        st.error("Please enter a URL.")
    else:
        with st.spinner("Fetching and analyzing..."):
            try:
                html = fetch_html(url)
                text = extract_main_text(html)
                features = extract_features_from_text(text)
                label, probs = scorer.predict_label(features)  # label str, probs dict
            except Exception as e:
                st.error(f"Failed to analyze URL: {e}")
                st.stop()

        # Show results
        st.markdown("### Quality Summary")
        st.write({
            "URL": url,
            "Title": features.get("title", ""),
            "Word count": features["wordcount"],
            "Readability (Flesch)": features["flesch_reading_ease"],
            "Avg sentence len": round(features["avg_sentence_length"], 2),
            "Lexical diversity": round(features["lexical_diversity"], 3),
            "Quality label": label
        })

        # Show text sample
        st.markdown("### Extracted Text (first 800 chars)")
        st.text(text[:800] + ("..." if len(text) > 800 else ""))

        # Similarity checks (if corpus available)
        if isinstance(corpus_df, pd.DataFrame) and len(corpus_df) > 0:
            st.markdown("### Similarity / Duplicate Check")
            corpus_texts = corpus_df['text'].fillna("").astype(str).tolist()
            corpus_urls = corpus_df['url'].fillna("").astype(str).tolist() if 'url' in corpus_df.columns else [f"row_{i}" for i in range(len(corpus_texts))]

            # build TF-IDF
            try:
                vect, X_corpus = build_tfidf(corpus_texts)
                X_q = vect.transform([text])
                if X_q.nnz == 0:
                    st.info("Query produced empty TF-IDF vector (too short or vocabulary mismatch). Try lower threshold or use embeddings.")
                else:
                    sims = cosine_similarity(X_q, X_corpus).flatten()
                    idx = np.argsort(-sims)[:top_k]
                    matches = [{"url": corpus_urls[i], "similarity": float(round(float(sims[i]),4)), "index": int(i)} for i in idx]
                    st.table(pd.DataFrame(matches))
                    above = [m for m in matches if m["similarity"] >= similarity_threshold]
                    if above:
                        st.success(f"{len(above)} matches above threshold ({similarity_threshold})")
                        st.table(pd.DataFrame(above))
                    else:
                        st.info(f"No matches exceeded the threshold {similarity_threshold}. Showing top {len(matches)} candidates.")
            except Exception as e:
                st.error(f"Similarity check failed: {e}")
        else:
            st.info("No corpus loaded for similarity checks (enable sidebar option or upload).")

st.markdown("---")
st.markdown("Notes: uses TF-IDF similarity and simple readability & length rules. For semantic similarity use precomputed embeddings.")


ModuleNotFoundError: No module named 'utils'